# Monarch

This notebook explores the biomedical knowledge graph provided by the project **Monarch**: [Publication (2024)](https://doi.org/10.1093/nar/gkad1082), [Website](https://monarchinitiative.org/), [Code](https://github.com/monarch-initiative/monarch-ingest), [Data](https://data.monarchinitiative.org/monarch-kg/index.html)

The source file of this notebook is [monarch.ipynb](https://github.com/robert-haas/awesome-biomedical-knowledge-graphs/blob/main/src/notebooks/monarch.ipynb) and can be found in the repository [awesome-biomedical-knowledge-graphs](https://github.com/robert-haas/awesome-biomedical-knowledge-graphs) that also contains information about similar projects.

## Table of contents

1. [Setup](#1.-Setup)
2. [Data download](#2.-Data-download)
3. [Data import](#3.-Data-import)
4. [Data inspection](#4.-Data-inspection)
5. [Schema discovery](#5.-Schema-discovery)
6. [Knowledge graph reconstruction](#6.-Knowledge-graph-reconstruction)
7. [Subgraph exploration](#7.-Subgraph-exploration)

## 1. Setup

This section prepares the environment for the following exploratory data analysis.

### a) Import packages

From the [Python standard library](https://docs.python.org/3/library/index.html).

In [ ]:
import os

From the [Python Package Index (PyPI)](https://pypi.org).

In [ ]:
import dask.dataframe as dd
import gravis as gv
import igraph as ig

From a local Python module named [shared_bmkg.py](https://github.com/robert-haas/awesome-biomedical-knowledge-graphs/blob/main/src/notebooks/shared_bmkg.py). The functions in it are used in several similar notebooks to reduce code repetition and to improve readability.

In [ ]:
import shared_bmkg

### b) Create data directories

The raw data provided by the project and the transformed data generated throughout this notebook are stored in separate directories. If the notebook is run more than once, the downloaded data is reused instead of fetching it again, but all data transformations are rerun.

In [ ]:
project_name = "monarch"
download_dir = os.path.join(project_name, "downloads")
results_dir = os.path.join(project_name, "results")

shared_bmkg.create_dir(download_dir)
shared_bmkg.create_dir(results_dir)

## 2. Data download

This section fetches the data published by the project in a [data repository on a custom web server](https://data.monarchinitiative.org/monarch-kg/index.html). The latest available version at the time of creating this notebook was used: `2024-07-12`.

### All files provided by the project

Monarch provides regular updates of its data and provides it in many different formats.

- TSV files
  - `monarch-kg.tar.gz`: [Tarball](https://en.wikipedia.org/wiki/Tar_(computing)) archive with two files
    - `monarch-kg_nodes.tsv`: Nodes and node annotations
    - `monarch-kg_edges.tsv`: Edges and edge annotations
  - `monarch-kg-denormalized-edges.tsv.gz`
  - `monarch-kg-denormalized-nodes.tsv.gz`
- Database dumps
  - `monarch-kg.db.gz`: [SQLite](https://en.wikipedia.org/wiki/SQLite) relational database
  - `monarch-kg.duckdb.gz`: [DuckDB](https://en.wikipedia.org/wiki/DuckDB) relational database
  - `monarch-kg.neo4j.dump`: [Neo4j](https://en.wikipedia.org/wiki/Neo4j) graph database
- RDF files
  - `monarch-kg.jsonl.tar.gz`: [JSON-LD](https://en.wikipedia.org/wiki/JSON-LD) file
  - `monarch-kg.nt.gz`: [N-Triples](https://en.wikipedia.org/wiki/N-Triples) file
- Metadata
  - `metadata.yaml`
  - `merged_graph_stats.yaml`
  - `qc_report.yaml`
- Other
  - `phenio.db.gz`: Phenomics Integrated Ontology database
  - `solr.tar.gz`: Solr index for the website's search function

Remarks in the publication:
- "For users who want to integrate Monarch’s KG into their own datasets or analysis pipelines, we use Knowledge Graph Exchange (KGX) tools to serialize to various formats (e.g. SQLite, Neo4J, RDF, KGX)."
- "the Monarch Knowledge Graph (KG), which comprises the combined knowledge of 33 biomedical resources and biomedical ontologies (see Data Sources below), and is updated with the latest data from each source once a month."
- "ontologies are integrated into a ‘semantic layer,’ a Biolink-conformant representation of the Phenomics Integrated Ontology (PHENIO; github.com/monarch-initiative/phenio), which serves as a hierarchical schema and classification system for the integrated data."

### Files needed to create the knowledge graph

- `monarch-kg.tar.gz` contains all information required for reconstructing the knowledge graph.
- Alternatively, many other formats are provided and could be used instead.

In [ ]:
download_specification = [
    # TAR archive of two files that contain the knowledge graph and annotations for nodes and edges
    # - monarch-kg_nodes.tsv: TSV file that contains the node data
    # - monarch-kg_edges.tsv: TSV file that contains the edge data
    ("monarch-kg.tar.gz", "https://data.monarchinitiative.org/monarch-kg/2024-07-12/monarch-kg.tar.gz", "df24db3c3b743c5829af71b6cd41c9fb"),
]

for filename, url, md5 in download_specification:
    filepath = os.path.join(download_dir, filename)
    shared_bmkg.fetch_file(url, filepath)
    shared_bmkg.validate_file(filepath, md5)
    print()

In [ ]:
filepath_extracted_1 = os.path.join(download_dir, "monarch-kg_nodes.tsv")
filepath_extracted_2 = os.path.join(download_dir, "monarch-kg_edges.tsv")

if os.path.isfile(filepath_extracted_1) and os.path.isfile(filepath_extracted_2):
    print("Found existing files from a previous extraction of the archive.")
else:
    filepath = os.path.join(download_dir, "monarch-kg.tar.gz")
    shared_bmkg.extract_tar_gz(filepath)

## 3. Data import

This section loads the raw files into Python data structures for the following inspection and conversion.

In [ ]:
def read_tsv_file(filepath):
    with open(filepath) as f:
        # A Dask dataframe, not Pandas
        df = dd.read_csv(filepath, sep='\t', dtype=str)
    return df

In [ ]:
%%time

df_nodes = read_tsv_file(os.path.join(download_dir, "monarch-kg_nodes.tsv"))
df_edges = read_tsv_file(os.path.join(download_dir, "monarch-kg_edges.tsv"))

## 4. Data inspection

This section attempts to reproduce some published numbers by inspecting the raw data and then prints a few exemplary records.

The file [merged_graph_stats.yaml](https://data.monarchinitiative.org/monarch-kg/2024-06-10/merged_graph_stats.yaml) of the 2024-06-10 release mentions following statistics about the knowledge graph contents, which can be found in the entries `total_nodes`, `total_edges`, `predicates` (number of entries), `node_categories` (number of entries):
- 1,028,155 nodes having 80 different node types
- 11,076,689 edges having 28 different edge types


### a) Number of nodes and edges

In [ ]:
%%time

num_nodes = len(df_nodes)
num_edges = len(df_edges)

print(f"{num_nodes:,} nodes")
print(f"{num_edges:,} edges")
print()

Interpretation:
- Inspecting the raw data resulted in **1,028,155 nodes**, which matches the number mentioned in the stats file.
- Inspecting the raw data resulted in **11,076,689 edges**, which matches the number mentioned in the stats file.

### b) Types of nodes and edges

In [ ]:
%%time

nt_column = "category"
nt_counts = df_nodes.groupby(nt_column).size().compute()
nt_counts = nt_counts.sort_values(ascending=False)

print(len(nt_counts), "node types, sorted by their frequency of occurrence:")
for type, cnt in sorted(nt_counts.items(), key=lambda item: -item[1]):
    print(f"- {type}: {cnt}")
print()

In [ ]:
%%time

et_column = "predicate"
et_counts = df_edges.groupby(et_column).size().compute()
et_counts = et_counts.sort_values(ascending=False)

print(len(et_counts), "edge types, sorted by their frequency:")
for key, val in et_counts.items():
    print(f"- {key}: {val}")
print()

In [ ]:
# Correctness checks

# 1) Do the counts of different node types add up to the total number of nodes?
sum_node_types = nt_counts.sum()
assert sum_node_types == num_nodes, f"Node counts differ: {sum_node_types} != {num_nodes}"
print(f"{sum_node_types:,} = {num_nodes:,} nodes")

# 2) Do the counts of different edge types add up to the total number of edges?
sum_edge_types = et_counts.sum()
assert sum_edge_types == num_edges, f"Edge counts differ: {sum_edge_types} != {num_edges}"
print(f"{sum_edge_types:,} = {num_edges:,} edges")

Interpretation:
- Inspecting the raw data resulted in **80 node types**, which matches the number mentioned in the stats file.
- Inspecting the raw data resulted in **28 edge types**, which matches the number mentioned in the stats file.
- Looking at **relative frequencies of node and edge types** suggests that the dataset is rather unbalanced.
  - The most frequent node type is "biolink:Gene" with 571,074 instances, while the least frequent node types only have 1 instance each.
  - The most frequent edge type is "biolink:interacts_with" with 2,799,181 instances, while the least frequent edge type is "biolink:acts_upstream_of_or_within_negative_effect" with 180 instances, a difference of four orders of magnitude.

### c) Example entries

This section prints some example entries of the raw data. It gives an impression of the format chosen by the authors, which differs greatly between projects due to a lack of a broadly accepted standard for biomedical knowledge graphs.

In [ ]:
def report_first_n_items(data, n):
    return data.head(n)

In [ ]:
def report_last_n_items(data, n):
    return data.tail(n)

#### Nodes together with node annotations

In [ ]:
report_first_n_items(df_nodes, 2)

In [ ]:
report_last_n_items(df_nodes, 2)

#### Edges together with edge annotations

In [ ]:
report_first_n_items(df_edges, 2)

In [ ]:
report_last_n_items(df_edges, 2)

## 5. Schema discovery

This section analyzes the structure of the knowledge graph by determining which types of nodes are connected by which types of edges. To construct this overview, it is necessary to iterate over the entire data once. The result is a condensed representation of all entities and relations, which is known as [data model](https://neo4j.com/docs/getting-started/data-modeling/guide-data-modeling/#whiteboard-friendly) or [schema](https://memgraph.com/docs/fundamentals/graph-modeling#designing-a-graph-database-schema) in the context of graph databases.

In [ ]:
node_type_to_color = {
    "biolink:Drug": "green",
    "biolink:ChemicalEntity": "green",
    "biolink:MolecularEntity": "green",
    "biolink:SmallMolecule": "green",

    "biolink:Gene": "blue",
    "biolink:Protein": "blue",

    "biolink:Disease": "red",
    "biolink:Pathway": "red",
    "biolink:BiologicalProcessOrActivity": "red",
}

In [ ]:
%%time

node_id_to_type = {row.id: row.category for row in df_nodes.itertuples()}

In [ ]:
%%time

unique_triples = set()
for row in df_edges.itertuples():
    s = node_id_to_type[row.subject]
    p = row.predicate
    o = node_id_to_type[row.object]
    triple = (s, p, o)
    unique_triples.add(triple)

In [ ]:
gs = ig.Graph(directed=True)
unique_nodes = set()
for s, p, o in unique_triples:
    for node in (s, o):
        if node not in unique_nodes:
            unique_nodes.add(node)
            
            node_size = int(nt_counts[node])
            node_color = node_type_to_color.get(node, '')
            node_hover = f"{node}\n\n{nt_counts[node]} nodes of this type are contained in the knowledge graph."
            gs.add_vertex(node, size=node_size, color=node_color, label_color=node_color, hover=node_hover)

    edge_size = int(et_counts[p])
    edge_color = node_type_to_color.get(s, '')
    edge_hover = f"{p}\n\n{et_counts[p]} edges of this type are contained in the knowledge graph."
    gs.add_edge(s, o, size=edge_size, color=edge_color, hover=edge_hover, label=p, label_color="gray", label_size=5)

gs.vcount(), gs.ecount()

In [ ]:
fig = gv.d3(
    gs,
    show_node_label=True,
    node_label_data_source="name",

    show_edge_label=True,
    edge_label_data_source="label",
    edge_curvature=0.2,

    use_node_size_normalization=True,
    node_size_normalization_min=10,
    node_size_normalization_max=50,
    node_drag_fix=True,
    node_hover_neighborhood=True,
    
    use_edge_size_normalization=True,
    edge_size_normalization_max=3,

    many_body_force_strength=-3000,
    zoom_factor=0.3,
)
fig

In [ ]:
# Export the schema visualization
schema_filepath = os.path.join(results_dir, f"{project_name}_schema.html")
fig.export_html(schema_filepath, overwrite=True)

Interpretation:
- Each node in the schema corresponds to one of the 80 node types in the data.
  - *Node size* represents the number of instances, i.e. how often that node type is present in the knowledge graph. The exact number can also be seen when hovering over a node.
  - *Node color* represents particular node types. The coloring scheme is based on a deliberately simple RGB palette with the same meaning across multiple notebooks to enable some visual comparison. The idea behind it is to highlight an interplay of certain entities, namely that drugs (or small molecules in general) can bind to proteins (or gene products in general) and thereby alter diseases (or involved pathways).
    - *green* = drugs & other small molecules (e.g. toxins)
    - *blue* = genes & gene products (e.g. proteins or RNAs)
    - *red* = diseases & related concepts (e.g. pathways)
    - *black* = all other types of entities
- Each edge in the schema stands for one of the 28 edge types in the data. It is possible that the same edge type appears between different nodes.
  - *Edge size* represents the number of instances, i.e. how often that edge type is present in the knowledge graph.
  - *Edge color* is identical to the color of the source node, again to highlight the interplay between drugs, targets and diseases.

## 6. Knowledge graph reconstruction

This section first converts the raw data to an intermediate format used in several notebooks, and then reconstructs the knowledge graph from the standardized data with shared code.
- The intermediate form of the data is created as two simple Python lists, one for nodes and the other for edges, which can be exported to two CSV files.
- The knowledge graph is built as a graph object from the Python package [igraph](https://igraph.org/python), which can be exported to a [GraphML](https://en.wikipedia.org/wiki/GraphML) file.

### a) Convert the data into a standardized format

Transform the raw data to an standardized format that is compatible with most biomedical knowledge graphs in order to enable shared downstream processing:
- Each node is represented by three items: `id (str), type (str), properties (dict)`
- Each edge is represented by four items: `source_id (str), target_id (str), type(str), properties (dict)`

This format was initially inspired by a straightforward way in which the content of a Neo4j graph database can be exported to two CSV files, one for all nodes and the other for all edges. This is an effect of the [property graph model](https://neo4j.com/docs/getting-started/appendix/graphdb-concepts/) used in Neo4j and many other graph databases, which also appears to be general enough to fully capture the majority of biomedical knowledge graphs described in scientific literature, despite the large variety of formats they are shared in.

A second motivation was that each line represents a single node or edge, and that no entry is connected to any sections at other locations, such as property descriptions at the beginning of a GraphML file. This structural simplicity makes it very easy to load just a subset of nodes and edges by picking a subset of lines, or to skip the loading of properties if they not required for a task simply by ignoring a single column.

Finally, this format also allows to load the data directly into popular SQL databases like [SQLite](https://www.sqlite.org), [MySQL](https://www.mysql.com) or [PostgreSQL](https://www.postgresql.org/) with built-in CSV functions ([CSV in SQLite](https://www.sqlite.org/draft/cli.html#importing_files_as_csv_or_other_formats), [CSV in MySQL](https://dev.mysql.com/doc/refman/en/loading-tables.html), [CSV in PostgreSQL](https://www.postgresql.org/docs/current/sql-copy.html)). Further, the JSON string in the property column can be accessed directly by built-in JSON functions ([JSON in SQLite](https://www.sqlite.org/json1.html), [JSON in MySQL](https://dev.mysql.com/doc/refman/en/json-function-reference.html), [JSON in PostgreSQL](https://www.postgresql.org/docs/current/functions-json.html)), which enables sophisticated queries that access or modify specific properties within the JSON data.

#### Nodes

In [ ]:
%%time

nodes = []
for row in df_nodes.itertuples():
    node_id = row.id
    node_type = row.category
    node_properties = {
        "label": row.name,  # Caution: The attribute "name" is reserved in igraph as unique identifier of a node, therefore using "label"
        "description": row.description,
        "xref": row.xref,
        "provided_by": row.provided_by,
        "synonym": row.synonym,
        "full_name": row.full_name,
        "in_taxon": row.in_taxon,
        "in_taxon_label": row.in_taxon_label,
        "symbol": row.symbol,
        "deprecated": row.deprecated,
        "iri": row.iri,
        "same_as": row.same_as,
    }
    node = (node_id, node_type, node_properties)
    nodes.append(node)

#### Edges

In [ ]:
%%time

edges = []
for row in df_edges.itertuples():
    source_id = row.subject
    target_id = row.object
    edge_type = row.predicate
    edge_properties = {
        "id": row.id,
        "original_subject": row.original_subject,
        "original_object": row.original_object,
        "category": row.category,
        "agent_type": row.agent_type,
        "aggregator_knowledge_source": row.aggregator_knowledge_source,
        "knowledge_level": row.knowledge_level,
        "primary_knowledge_source": row.primary_knowledge_source,
        "qualifiers": row.qualifiers,
        "provided_by": row.provided_by,
        "has_evidence": row.has_evidence,
        "publications": row.publications,
        "stage_qualifier": row.stage_qualifier,
        "frequency_qualifier": row.frequency_qualifier,
        "has_count": row.has_count,
        "has_percentage": row.has_percentage,
        "has_quotient": row.has_quotient,
        "has_total": row.has_total,
        "negated": row.negated,
        "onset_qualifier": row.onset_qualifier,
        "sex_qualifier": row.sex_qualifier,
    }
    edge = (source_id, target_id, edge_type, edge_properties)
    edges.append(edge)

### b) Export the standardized data to two CSV files

Both the `id` and `type` items are simple strings, while the `properties` item is collection of key-value pairs represented by a Python dictionary that can be converted to a single JSON string, which the export function does internally. This means each node is fully represented by three strings, and each edge by four strings due to having a source id and target id.

In [ ]:
%%time

nodes_csv_filepath = shared_bmkg.export_nodes_as_csv(nodes, results_dir, project_name)

In [ ]:
%%time

edges_csv_filepath = shared_bmkg.export_edges_as_csv(edges, results_dir, project_name)

### c) Use the standardized data to build a graph

Reconstruct the knowledge graph in form of a [Graph object](https://igraph.org/python/doc/api/igraph.Graph.html) from the package [igraph](https://igraph.org/python). This kind of graph object allows to have directed multi-edges, i.e. an edge has a source and a target node, and two nodes can be connected by more than one edge. It also allows to have node and edge properties. These features are necessary and sufficient to represent almost any biomedical knowledge graph found in academic literature.

In [ ]:
%%time

g = shared_bmkg.create_graph(nodes, edges)

In [ ]:
shared_bmkg.report_graph_stats(g)

In [ ]:
# Correctness checks

# 1) Does the reconstructed graph contain the same number of nodes as the raw data?
num_nodes_in_graph = g.vcount()
assert num_nodes_in_graph == num_nodes, f"Node counts differ: {num_nodes_in_graph} != {num_nodes}"
print(f"{num_nodes_in_graph:,} = {num_nodes:,}")

# 2) Does the reconstructed graph contain the same number of (unique) edges as the raw data?
num_edges_in_graph = g.ecount()
assert num_edges_in_graph == num_edges, f"Edge counts differ: {num_edges_in_graph} != {num_unique_triples}"
print(f"{num_edges_in_graph:,} = {num_edges:,}")

### d) Export the graph to a GraphML file

Export the graph with all nodes, edges and properties as a single [GraphML](https://en.wikipedia.org/wiki/GraphML) file.

In [ ]:
%%time

g_graphml_filepath = shared_bmkg.export_graph_as_graphml(g, results_dir, project_name)

## 7. Subgraph exploration

This section explores small subgraphs of the knowledge graph in two ways: first by inspecting the direct neighborhood of a selected node, and second by finding shortest paths between two chosen nodes.

As a simple case study, the goal is to identify some nodes in the knowledge graph that are associated with the success story of the drug Imatinib, which was one of the first [targeted therapies](https://en.wikipedia.org/wiki/Targeted_therapy) against cancer. Detailed background information can for example be found in an article by the [National Cancer Institute](https://www.cancer.gov/research/progress/discovery/gleevec) and in a [talk by Brian Druker](https://www.ibiology.org/human-disease/imatinib-paradigm-targeted-cancer-therapies) who played a major role in the development of this paradigm-changing drug. To give a simplified summary, following biological entities and relationships are involved:

- Mutation: In a bone marrow stem cell, a translocation event between chromosome 9 and 22 leads to what has been called the [Philadelphia chromosome](https://en.wikipedia.org/wiki/Philadelphia_chromosome), which can be seen under a microscope and got named after the city it originally got discovered in.
- Gene: It turned out that this particular rearrangement of DNA fuses the [BCR](https://en.wikipedia.org/wiki/BCR_(gene)) gene on chromosome 22 to the [ABL1](https://en.wikipedia.org/wiki/ABL_(gene)) gene on chromosome 9, resulting in a new fusion gene known as BCR-ABL1.
- Disease: BCR-ABL1 acts as an oncogene, because it expresses a protein that is a defective [tyrosine kinase](https://en.wikipedia.org/wiki/Tyrosine_kinase) in a permanent "on" state, which leads to uncontrolled growth of certain white blood cells and their precursors, thereby driving the disease [Chronic Myelogenous Leukemia (CML)](https://en.wikipedia.org/wiki/Chronic_myelogenous_leukemia).
- Drug: [Imatinib (Gleevec)](https://en.wikipedia.org/wiki/Imatinib) was the first demonstration that a potent and selective [Bcr-Abl tyrosine-kinase inhibitor (TKI)](https://en.wikipedia.org/wiki/Bcr-Abl_tyrosine-kinase_inhibitor) is possible and that such a targeted inhibition of an oncoprotein halts the uncontrolled growth of leukemia cells with BCR-ABL1, while having significantly less effect on other cells in the body compared to conventional chemotherapies used in cancer. This revolutionized the treatment of CML and drastically improved the five-year survival rate of patients from less than 20% to over 90%, as well as their quality of life.

In reality the story is a bit more complex, for example because there are other genes involved in disease progression, there are many closely related forms of leukemia, BCR-ABL1 also plays a role in other forms of cancer, there are several drugs available as treatment options today, all of them bind to more than one target and with different affinities, and their individual binding profiles are relevant to their particular therapeutic effects. Inspecting the knowledge graph will focus on highlighting some entities of the simplified story, but the surrounding elements will also indicate some of the complexity encountered in reality. Some simple forms of reasoning on the knowledge graph will demonstrate its potential for discovering new patterns and hypotheses.

### a) Search for interesting nodes

In [ ]:
# Drug: Imatinib - seems not to be contained in Monarch
shared_bmkg.list_nodes_matching_substring(g, "imatinib", "label")

In [ ]:
# Gene: ABL1
shared_bmkg.list_nodes_matching_substring(g, "abl1", "label")

In [ ]:
# Disease: Chronic Myeloid Leukemia (CML)
shared_bmkg.list_nodes_matching_substring(g, "chronic myelogenous leukemia", "label")

### b) Explore the neighborhood of a chosen node

In [ ]:
# Neighborhood of gene ABL1 - in Homo sapiens
source = "HGNC:76"
subgraph = shared_bmkg.get_egocentric_subgraph(g, source)

# Export
filename = f"{project_name}_neighbors_abl1"
shared_bmkg.export_graph_as_graphml(subgraph, results_dir, filename)
shared_bmkg.export_nodes_as_csv(nodes, results_dir, filename, subgraph)
shared_bmkg.export_edges_as_csv(edges, results_dir, filename, subgraph)

# Report
shared_bmkg.report_graph_stats(subgraph)

Interpretation:
- Monarch contains too many nodes connected to the gene ABL1 for plotting it and performing a visual analysis.

In [ ]:
# Neighborhood of disease CML
source = "MONDO:0011996"
subgraph = shared_bmkg.get_egocentric_subgraph(g, source)

# Export
filename = f"{project_name}_neighbors_cml"
shared_bmkg.export_graph_as_graphml(subgraph, results_dir, filename)
shared_bmkg.export_nodes_as_csv(nodes, results_dir, filename, subgraph)
shared_bmkg.export_edges_as_csv(edges, results_dir, filename, subgraph)

# Report
shared_bmkg.report_graph_stats(subgraph)
shared_bmkg.visualize_graph(subgraph, node_type_to_color, source)

Interpretation:
- Disease-Disease relations: The disease CML with id `MONDO:0011996` (red node in the center) stands in a "biolink:subclass_of" relation to many other diseases (red nodes). The type of relation can be seen when hovering over an arrow. The schema also summarized the edge types found between pairs of node types. In this case, there can be "biolink:subclass_of" and "biolink:related_to" edges between pairs of diseases.
  - Example: The disease `MONDO:0010809` (="familial chronic myelocytic leukemia-like syndrome") is a subclass of the disease CML with id `MONDO:0011996` (="chronic myelogenous leukemia, BCR-ABL1 positive"), which in turn is a subclass of the disease `MONDO:0004643` (=myeloid leukemia).
- Disease-Gene relations: The disease CML stands in "biolink:gene_associated_with_condition" and "biolink:causes" relations with genes (blue nodes).
  - Example: The two genes `HGNC:1014` (="BCR") and `HGNC:76` (="ABL1") are connected with both these relations to the disease CML, while the gene `HGNC:10471` (="RUNX1") only stands in the weaker "biolink:gene_associated_with_condition" relation to CML.
- Other relations: The disease CML stands in various relations to node types like "biolink:Genotype" or "biolink:PhenotypicFeature" (black nodes).
  - Example: The disease CML stands in a "biolink:has_phenotype" relation to the node `HP:0001894` (="Thrombocytosis"), which is described as "Increased numbers of platelets in the peripheral blood.". This information can be read when hovering over the edges or nodes in this visualization.

### c) Find shortest paths between two chosen nodes

In [ ]:
# Paths from gene ABL1 to disease "myeloid leukemia"
source = "HGNC:76"
target = "MONDO:0004643"
subgraph = shared_bmkg.get_paths_subgraph(g, source, target)

# Export
filename = f"{project_name}_paths_abl1_to_ML"
shared_bmkg.export_graph_as_graphml(subgraph, results_dir, filename)
shared_bmkg.export_nodes_as_csv(nodes, results_dir, filename, subgraph)
shared_bmkg.export_edges_as_csv(edges, results_dir, filename, subgraph)

# Report
shared_bmkg.report_graph_stats(subgraph)
shared_bmkg.visualize_graph(subgraph, node_type_to_color, source, target)

Interpretation:
- The gene `HGNC:76` (="ABL1") is connected to the disease `MONDO:0004643` (="myeloid leukemia") only by one shortest path via the disease `MONDO:0011996` (="chronic myelogenous leukemia, BCR-ABL1 positive"). This path was already part of a previous subgraph showing the neighborhood of CML, but there could have been alternative paths as well.